In [ ]:
!nvidia-smi

#**TIẾN HÀNH DOWNLOAD DATA VỀ**

In [ ]:
# Mount drive CỦA CÁ NHÂN
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import gdown
%cd /content/gdrive/My\ Drive
!mkdir Data_CS114
%cd /content/gdrive/My\ Drive/Data_CS114

In [ ]:
!gdown --id 1FRg4q4qTi2ImfpMxE5EnLeDRCHNxo13C #Testimg.zip
!gdown --id 1gDtyugP41svdzTfWaWyysWC4Co3X0rIs #Testlabel.zip

In [ ]:
!mkdir Test

In [ ]:
#UNZIP data
!unzip testimg.zip -d "/content/gdrive/MyDrive/Data_CS114/Test"

!unzip testlabel.zip -d "/content/gdrive/MyDrive/Data_CS114/Test"

In [ ]:
# import các thư viện cần thiết
import urllib
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader, Dataset
import os 
from PIL import Image
import random
import xml.etree.ElementTree as ET
import time
import requests
from google.colab.patches import cv2_imshow
import matplotlib.patches as patches

In [ ]:
# path of images directory
img_test_path = '/content/gdrive/MyDrive/Data_CS114/Test/testimg'

# path of xml files directory
label_test_path = '/content/gdrive/MyDrive/Data_CS114/Test/test_label'

# List of Image file name 

img_test_list = list(sorted(os.listdir(img_test_path)))
label_test_list= list(sorted(os.listdir(label_test_path)))

# How many image files?
#print('There are total {} images train and {} images test.'.format(len(img_train_list),len(img_test_list)))

#**HÀM ĐỌC LABEL VÀ VẼ BOUNDING BOX**

In [ ]:
# Helper function for read the data (label and bounding boxes) from xml file 
def read_label(file_name, label_dir):
    """
    Function used to get the bounding boxes and labels from the xml file
    Input:
        file_name: image file name
        xml_dir: directory of xml file
    Return:
        bbox : list of bounding boxes
        labels: list of labels
    """
    bbox = []
    labels = []
    
    label_path = os.path.join(label_dir, file_name[:-3]+'xml')
    tree = ET.parse(label_path)
    root = tree.getroot()
    for boxes in root.iter('object'):
        ymin = int(boxes.find("bndbox/ymin").text)
        xmin = int(boxes.find("bndbox/xmin").text)
        ymax = int(boxes.find("bndbox/ymax").text)
        xmax = int(boxes.find("bndbox/xmax").text)
        label = boxes.find('name').text
        bbox.append([xmin,ymin,xmax,ymax])
        if label == 'Correct':
            label_idx = 1
        elif label == 'No-mask':
          label_idx = 2
        elif label == 'Incorrect':
            label_idx = 3
        labels.append(label_idx)
        
    return bbox, labels

# help function for drawing bounding boxes on image
def draw_boxes(img, boxes,labels, thickness=3):
    """
    Function to draw bounding boxes
    Input:
        img: array of img (h, w ,c)
        boxes: list of boxes (int)
        labels: list of labels (int)
    
    """
    font =  cv2.FONT_HERSHEY_SIMPLEX
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    for box,label in zip(boxes,labels):
        box = [int(x) for x in box]
        if label == 3:
          color = (255,0,255) #pink
          text = 'Incorrect'
        elif label == 2:
          color = (0,0,225) # red
          text = 'Nomask'
        elif label == 1:
          color = (0,255,0) # green
          text = 'Correct'
        cv2.putText(img,text,(box[0],box[1]), font, 1 ,color, thickness)
        cv2.rectangle(img, (box[0],box[1]),(box[2],box[3]),color,thickness)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#**THIẾT LẬP DATASET**

In [ ]:
class image_dataset(Dataset):
    def __init__(self, image_list, image_dir, label_train_dir):
        self.image_list = image_list
        self.image_dir = image_dir
        self.xml_dir = label_train_dir
        self.classes = [_, 'Correct','No-mask','Incorrect']
    def __getitem__(self, idx):
        """
        Load the image
        """
        img_name = self.image_list[idx]
        img_path = os.path.join(self.image_dir, img_name)
        img = Image.open(img_path).convert('RGB')
        img = transforms.ToTensor()(img)

        """
        build the target dict
        """
        bbox, labels = read_label(img_name, self.xml_dir)
        boxes = torch.as_tensor(bbox, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        
        area = (boxes[:,2] - boxes[:,0]) * (boxes[:,3] - boxes[:,1])
        area = torch.as_tensor(area, dtype=torch.float32)
        iscrowd = torch.zeros((len(bbox),), dtype=torch.int64)
        
        target = {}
        
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([idx])
        target['area'] = area
        target['iscrowed'] = iscrowd
        return img , target
                    
    def __len__(self):
        return len(self.image_list)

In [ ]:

mask_dataset_test = image_dataset(img_test_list, img_test_path, label_test_path)

def collate_fn(batch):
    return tuple(zip(*batch))


mask_loader_test = DataLoader(mask_dataset_test,batch_size=10,shuffle=True,num_workers=4,collate_fn=collate_fn)

#**THIẾT LẬP CÁC THAM SỐ**

In [ ]:
def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
%cd /content/gdrive/My\ Drive/Data_CS114
!gdown --id 1UaAA1bB3xkwzqtbSK639R0xbeD5bXAeh

In [ ]:
model = get_model_instance_segmentation(4)
model.load_state_dict(torch.load('/content/gdrive/MyDrive/Data_CS114/model_final_cs114_1.pth'))
model = model.to(device)

#**PREDICT MODEL**

In [ ]:
def single_img_predict(img, nm_thrs = 0.3, score_thrs=0.5):
    test_img = transforms.ToTensor()(img)
    model.eval()
    
    with torch.no_grad():
        predictions = model(test_img.unsqueeze(0).to(device))
        
    test_img = test_img.permute(1,2,0).numpy()
    
    # non-max supression
    keep_boxes = torchvision.ops.nms(predictions[0]['boxes'].cpu(),predictions[0]['scores'].cpu(),nm_thrs)
    
    # Only display the bounding boxes which higher than the threshold
    score_filter =  predictions[0]['scores'].cpu().numpy()[keep_boxes] > score_thrs
    # get the filtered result
    test_boxes = predictions[0]['boxes'].cpu().numpy()[keep_boxes][score_filter]
    test_labels = predictions[0]['labels'].cpu().numpy()[keep_boxes][score_filter]
    
    return test_img, test_boxes, test_labels

In [ ]:
    idx = random.randint(1,len(img_test_list))
    # idx = 210
    test_img = Image.open(os.path.join(img_test_path,img_test_list[idx]))
    # test_img = test_img.cuda()
    # Prediction
    test_img, test_boxes, test_labels= single_img_predict(test_img)
    # print(test_boxes)
    # print(test_labels)
    # print(score[:len(test_labels)])
    test_output = draw_boxes(test_img, test_boxes,test_labels)
    # t = test_output

    # Draw the bounding box of ground truth
    bbox, labels  = read_label(img_test_list[idx], label_test_path)
    # #draw bounding boxes on the image
    gt_output = draw_boxes(test_img, bbox,labels)
    print(idx)
    # img1 = Image.fromarray((test_output * 150).astype(np.uint8))
    # img1 = img1.convert('RGB')
    # img1.save("/content/drive/MyDrive/Data_Project_CS114/Predict/imagePred{}.jpg".format(idx))
    # Display the result
    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(40,80))
    ax1.imshow(test_output)
    ax1.set_xlabel('Prediction')
    ax2.imshow(gt_output)
    ax2.set_xlabel('Ground Truth')
    plt.show()